# Temporal Pres

In [1]:
using CSV, DataFrames, Statistics, Random, ParallelKMeans, Gurobi, JuMP

In [3]:
df = CSV.read("data/pred_temporal.csv", DataFrame)
df;

In [4]:
df_no_missing = dropmissing(df);

In [160]:
df_no_missing[:, 1:10]

Row,Column1,State,Date,population,confirmed,deaths,incident_rate,mortality_rate,testing_rate,TestsReported
,Int64,String31,Date,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,52,Alabama,2020-04-13,0.111072,0.00500903,0.00300674,0.0158113,0.272074,0.0118867,2626
2,53,Alaska,2020-04-13,0.00392428,1.01222e-5,0.000242969,0.00508025,0.296715,0.0266052,399
3,54,Arizona,2020-04-13,0.172087,0.0049671,0.00370528,0.00655017,0.337782,0.0113124,947
4,55,Arkansas,2020-04-13,0.0626465,0.00164847,0.000880763,0.00769774,0.211499,0.0157655,293
5,56,California,2020-04-13,1.0,0.0342144,0.021685,0.00981881,0.306982,0.00898563,9150
6,57,Colorado,2020-04-13,0.133047,0.010731,0.00929357,0.033892,0.408624,0.0127428,50
7,58,Connecticut,2020-04-13,0.0767085,0.0189588,0.0182834,0.111124,0.462012,0.0251683,1680
8,59,Delaware,2020-04-13,0.0101456,0.00215169,0.00124522,0.0483397,0.23922,0.0245488,767
9,60,District of Columbia,2020-04-13,0.00326172,0.00243655,0.0015793,0.0794369,0.273101,0.0317294,391


In [104]:
function exponential_decay_weights(t, decay_rate)
    time_points = 1:t
    res = [exp(-decay_rate * x) for x in time_points]
    somme = sum(res)
    return reverse([x/somme for x in res])
end

exponential_decay_weights (generic function with 1 method)

In [47]:
pred = [eval(Meta.parse(x)) for x in df_no_missing[:, "Value"]]
neigh = [eval(Meta.parse(x)) for x in df_no_missing[:, "Neighbors"]];

7124-element Vector{Vector{Vector{Int64}}}:
 [[0, 41, 25, 20]]
 [[1, 19, 29, 34]]
 [[2, 20, 25, 47]]
 [[3, 27, 31, 12]]
 [[4, 44, 9, 38]]
 [[5, 50, 14, 23]]
 [[6, 18, 28, 16]]
 [[7, 40, 29, 34]]
 [[8, 40, 7, 29]]
 [[9, 44, 38, 33]]
 [[10, 35, 13, 2]]
 [[11, 12, 26, 31]]
 [[12, 11, 31, 27]]
 ⋮
 [[18, 30, 6, 21], [70, 82, 58, 73, 60], [134, 122, 110, 125, 112], [186, 174, 177, 162], [238, 226, 229, 214, 216], [290, 278, 281, 300, 266], [342, 330, 333, 352, 318], [394, 382, 385, 404], [446, 434, 456, 437, 422], [498, 508, 486, 489]  …  [6644, 6612, 6622, 6635, 6611], [6696, 6664, 6687, 6674, 6663], [6748, 6716, 6739, 6726, 6715], [6800, 6768, 6791, 6778, 6767], [6852, 6820, 6843, 6830, 6819], [6904, 6872, 6895, 6882, 6871], [6956, 6924, 6947, 6934, 6923], [7008, 6976, 6999, 6986, 6975], [7060, 7028, 7051, 7038, 7027], [7112, 7080, 7103, 7090, 7079]]
 [[30, 18, 32, 21], [82, 70, 73, 84, 58], [134, 125, 122, 136, 144], [186, 177, 174, 188], [238, 229, 248, 226, 216], [290, 281, 300, 278, 26

In [86]:
function get_matrix(liste_w, liste_y)
    n = length(liste_w)
    matrix_w = zeros(Float64, n, 52)
    matrix_y = zeros(Float64, n, 52)
    for time in 1:n
        neighbors = liste_w[time]
        values = liste_y[time]
        k = length(neighbors)
        for (ind,x) in enumerate(neighbors)
            x = x - 52*(time-1)
            matrix_w[time, x+1] = 1/k
            matrix_y[time, x+1] = values[ind]
        end
    end
    return matrix_w, matrix_y
end

get_matrix (generic function with 2 methods)

In [133]:
get_matrix(neigh[1], pred[1])[2]

1×52 Matrix{Float64}:
 2165.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [151]:
env = Gurobi.Env();
function optimisation_cost(ind, decay_rate, solver_output=0)
    matrix_w, matrix_y = get_matrix(neigh[ind], pred[ind])
    time = size(matrix_w)[1]
    nb_states = size(matrix_w)[2]
    
    weights_temporal = exponential_decay_weights(time, decay_rate)
    
    # Build model
    model = Model(()-> Gurobi.Optimizer(env))
    set_optimizer_attribute(model, "OutputFlag", solver_output)
    
    # Variables
    @variable(model, z >= 0)
    
    @variable(model, s[t in 1:time, k in 1:nb_states])
    
    # Constraint
    @constraint(model, [t in 1:time, k in 1:nb_states], s[t, k] >= - matrix_y[t, k])
    @constraint(model, [t in 1:time, k in 1:nb_states], s[t, k] >= - z)
    # Objective
    #@objective(model, Min,  sum(weights_temporal[t]* sum(matrix_w[t, k]*(2*z + 10*s[t, k]) for k in 1:nb_states) for t in 1:time))
    @objective(model, Min,  sum(weights_temporal[t]*matrix_w[t, k]*(2*z + 10*s[t, k]) for k in 1:nb_states, t in 1:time))
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    
    return (value.(z))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-23


optimisation_cost (generic function with 2 methods)

In [149]:
optimisation_cost(80, 1, 1)

[0.2689414213699951, 0.7310585786300049]
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 208 rows, 105 columns and 312 nonzeros
Model fingerprint: 0x16c65ce5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 2e+03]
Presolve removed 199 rows and 95 columns
Presolve time: 0.00s
Presolved: 9 rows, 10 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.4196473e+03   3.657500e+02   0.000000e+00      0s
       3   -2.8047951e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective -2.804795116e+03

User-callback calls 46, time in user-callback 0.00 sec


853.0

In [161]:
new_pres = []
for ind in 1:length(pred)
    push!(new_pres, optimisation_cost(ind, 1, 0))
end

In [164]:
copy = df_no_missing[:, 1:10]
copy[:, "Pres"] = new_pres
copy

Row,Column1,State,Date,population,confirmed,deaths,incident_rate,mortality_rate,testing_rate,TestsReported,Pres
,Int64,String31,Date,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Any
1,52,Alabama,2020-04-13,0.111072,0.00500903,0.00300674,0.0158113,0.272074,0.0118867,2626,3015.0
2,53,Alaska,2020-04-13,0.00392428,1.01222e-5,0.000242969,0.00508025,0.296715,0.0266052,399,404.0
3,54,Arizona,2020-04-13,0.172087,0.0049671,0.00370528,0.00655017,0.337782,0.0113124,947,3015.0
4,55,Arkansas,2020-04-13,0.0626465,0.00164847,0.000880763,0.00769774,0.211499,0.0157655,293,1921.0
5,56,California,2020-04-13,1.0,0.0342144,0.021685,0.00981881,0.306982,0.00898563,9150,10816.0
6,57,Colorado,2020-04-13,0.133047,0.010731,0.00929357,0.033892,0.408624,0.0127428,50,2790.0
7,58,Connecticut,2020-04-13,0.0767085,0.0189588,0.0182834,0.111124,0.462012,0.0251683,1680,2703.0
8,59,Delaware,2020-04-13,0.0101456,0.00215169,0.00124522,0.0483397,0.23922,0.0245488,767,1059.0
9,60,District of Columbia,2020-04-13,0.00326172,0.00243655,0.0015793,0.0794369,0.273101,0.0317294,391,1059.0
